In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import os

In [2]:
"""
Задание 1**
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг
Результат классификации запишите в столбец class
"""


def range_rates(row):
    if row.rating <= 2:
        return "низкий рейтинг"
    elif 2 < row.rating < 4.5:
        return "средний рейтинг"
    elif row.rating >= 4.5:
        return "высокий рейтинг"


df = pd.read_csv(filepath_or_buffer=os.path.join(Path.home(), "ratings.csv"))
df['range_rating'] = df.apply(range_rates, axis=1)
print(df.loc[10:20])

    userId  movieId  rating   timestamp     range_rating
10       1     1371     2.5  1260759135  средний рейтинг
11       1     1405     1.0  1260759203   низкий рейтинг
12       1     1953     4.0  1260759191  средний рейтинг
13       1     2105     4.0  1260759139  средний рейтинг
14       1     2150     3.0  1260759194  средний рейтинг
15       1     2193     2.0  1260759198   низкий рейтинг
16       1     2294     2.0  1260759108   низкий рейтинг
17       1     2455     2.5  1260759113  средний рейтинг
18       1     2968     1.0  1260759200   низкий рейтинг
19       1     3671     3.0  1260759117  средний рейтинг
20       2       10     4.0   835355493  средний рейтинг


In [4]:
"""
Задание 2
Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую
принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона,
то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города,
то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {
'Центр': ['москва', 'тула', 'ярославль'],
'Северо-Запад': ['петербург', 'псков', 'мурманск'],
'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

Результат классификации запишите в отдельный столбец region.
"""

def revert_dict(data, inplace=False):
    res = {}
    
    for k,v in data.items():
        if type(v) != list:
            v = [v]
            
        for vi in v:
            if vi in res:
                raise ValueError('Value duplication detected ({})!'.format(vi))
            res[vi] = k

    if inplace:
        data.clear()
        data.update(res)
    
    return res

def apply_region(keywords_str):
    keywords = keywords_str.lower().split(' ')
    
    for keyword in keywords:
        if keyword in keyword_to_region_dict:
            return keyword_to_region_dict[keyword]
    
    return 'undefined'


geo_data = {
    'Центр':          ['москва', 'тула', 'ярославль'],
    'Северо-Запад':   ['петербург', 'псков', 'мурманск'],
    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

keywords_df = pd.read_csv('keywords.csv')
keyword_to_region_dict = revert_dict(geo_data)

keywords_df['region'] = keywords_df['keyword'].apply(apply_region)
grouped = keywords_df.groupby('region').count().reset_index()[['region', 'keyword']]
grouped.columns = ['Регион', 'Число']
grouped.sort_values('Число', ascending=False)


,Регион,Число
0,undefined,99406
3,Центр,384
2,Северо-Запад,146
1,Дальний Восток,64


In [17]:
"""
Задание 3
Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение,
используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл movies.csv и ratings.csv из 
базы grouplens.org.../movielens). 
Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.
При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:
В переменную years запишите список из всех годов с 1950 по 2010.
Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска.
Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
для каждой строки пройдите по всем годам списка years
если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’
Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга
"""

def production_year(title):
    for y in years:
        if str(y) in title:
            return y
    return 1900


movies_df = pd.read_csv(filepath_or_buffer=os.path.join(Path.home(), "movies.csv"))
ratings_df = pd.read_csv(filepath_or_buffer=os.path.join(Path.home(), "ratings.csv"))
years = range(1950, (2010 + 1))

movies_df['year'] = movies_df['title'].apply(production_year)
joined = ratings_df.merge(movies_df, on='movieId', how='left')

grouped = joined.groupby('year').agg({'rating': ['mean']})
grouped.columns = grouped.columns.droplevel(0)
grouped = grouped.reset_index()
sorted_by_mean = grouped.sort_values('mean', ascending=False)
sorted_by_mean

,year,mean
8,1957,4.014241
23,1972,4.011136
3,1952,4.000000
5,1954,3.994220
2,1951,3.983539
...,...,...
56,2005,3.448434
54,2003,3.444777
47,1996,3.426600
48,1997,3.415764
